In [3]:
import pandas as pd
import numpy as np
import zipfile
from IPython.display import display

In [2]:
# import data
zf = zipfile.ZipFile('../content/anonymisedData.zip') 

registrations = pd.read_csv(zf.open('studentRegistration.csv'))
courseInfo = pd.read_csv(zf.open('courses.csv'))
students = pd.read_csv(zf.open('studentInfo.csv'))
student_vle = pd.read_csv(zf.open('studentVle.csv'))
vle_info = pd.read_csv(zf.open('vle.csv'))
student_assessments = pd.read_csv(zf.open('studentAssessment.csv'), skiprows=[128223,64073])
assessments_info = pd.read_csv(zf.open('assessments.csv'))

index_columns = ['code_module','code_presentation','id_student']

In [50]:
    
    # import data
    zf = zipfile.ZipFile('../content/anonymisedData.zip') 

    registrations = pd.read_csv(zf.open('studentRegistration.csv'))
    courseInfo = pd.read_csv(zf.open('courses.csv'))
    students = pd.read_csv(zf.open('studentInfo.csv'))
    student_vle = pd.read_csv(zf.open('studentVle.csv'))
    vle_info = pd.read_csv(zf.open('vle.csv'))
    student_assessments = pd.read_csv(zf.open('studentAssessment.csv'), skiprows=[128223,64073])
    assessments_info = pd.read_csv(zf.open('assessments.csv'))

    index_columns = ['code_module','code_presentation','id_student']
    # Registrations
    full_registrations = pd.merge(students, registrations, on=index_columns, validate='1:1')
    full_registrations = pd.merge(full_registrations, courseInfo, \
                         on=['code_module','code_presentation'], validate='many_to_one')
    full_registrations.dropna(subset=['date_registration','imd_band'], inplace=True)

    not_withdrawn = full_registrations['date_unregistration'].isna()

    if type(prediction_window) == float:
        prediction_window = prediction_window*(full_registrations.module_presentation_length.min())
    if prediction_window:
        withdrawn_after_predict = (full_registrations['final_result'] == 'Withdrawn') \
                                    & (full_registrations['date_unregistration'] > prediction_window)
    else:
        withdrawn_after_predict = (full_registrations['final_result'] == 'Withdrawn') \
                                    & (full_registrations['date_unregistration'] > 0)
    full_registrations = full_registrations[not_withdrawn | withdrawn_after_predict]
    full_registrations['date_unregistration'].fillna( \
    full_registrations['module_presentation_length'], inplace = True)
    full_registrations = full_registrations[full_registrations['date_unregistration']
                                        <= full_registrations['module_presentation_length']]
    if prediction_window:
        full_registrations.drop(columns=['date_unregistration'], inplace=True)

    # VLE
    if prediction_window:
        student_vle = student_vle[student_vle.date <= prediction_window]
    vle = pd.merge(student_vle,vle_info, 
               how = 'left', \
               on =['id_site','code_module', 'code_presentation'], \
               validate = 'm:1').drop(columns = ['week_from','week_to'])

    total_activities = vle.groupby(by=index_columns).count().reset_index()
    total_activities = total_activities.drop(columns=['date','sum_click','activity_type'])

    date_grouped = vle.groupby(by=index_columns + ['date']).count().reset_index()
    days_studied = date_grouped.groupby(by=index_columns).count().reset_index()
    days_studied = days_studied.drop(columns=['id_site','sum_click','activity_type'])

    clicks = vle.groupby(by=index_columns).sum().reset_index()
    clicks = clicks.drop(columns=['id_site','date'])

    full_registrations = pd.merge(full_registrations, days_studied, on=index_columns, how='left')
    full_registrations = pd.merge(full_registrations, total_activities, on=index_columns, how='left')
    full_registrations = pd.merge(full_registrations, clicks, on=index_columns, how='left')


    # Assessments
    assessments = pd.merge(student_assessments, assessments_info, how='left', on='id_assessment')
    assessments.dropna(subset = ['score'], inplace=True)
    assessments.drop(columns = ['is_banked', 'weight'], inplace=True)
    if prediction_window:
        assessments = assessments[assessments['date_submitted'] <= prediction_window]
    num_assessments = assessments.groupby(by = index_columns).count().reset_index()
    total_assessments = assessments_info.groupby(by=['code_module','code_presentation']).count().reset_index()
    total_assessments = total_assessments.drop(columns = ['assessment_type','date','weight'])
    total_assessments = total_assessments.rename(columns = {'id_assessment':'total_assessments'})
    num_assessments = pd.merge(num_assessments, assess_count, 
                           how = 'left', on = ['code_module','code_presentation'])
    num_assessments['id_assessment'] = num_assessments['id_assessment']/num_assessments['total_assessments']
    num_assessments.drop(columns = ['total_assessments','date_submitted','score','date','assessment_type'], inplace=True)
    num_assessments
    avg_score = assessments.groupby(by = index_columns).mean().reset_index()
    avg_score.drop(columns = ['date_submitted','id_assessment','date'], inplace=True)

    full_registrations = pd.merge(full_registrations, num_assessments, on=index_columns, how='left')
    full_registrations = pd.merge(full_registrations, avg_score, on=index_columns, how='left')
    full_registrations = full_registrations.fillna(value=0)

    # Rename columns
    new_cols = {'id_assessment':'assessments_completed',
            'score':'average_assessment_score','date':'days_studied',
            'id_site':'activities_engaged','sum_click':'total_clicks'}
    full_registrations = full_registrations.rename(columns = new_cols)

    full_registrations

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,module_presentation_length,days_studied,activities_engaged,total_clicks,assessments_completed,average_assessment_score
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,268,26.0,121.0,628.0,0.500000,81.000000
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,268,45.0,266.0,896.0,0.333333,69.000000
2,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,268,68.0,371.0,1231.0,0.500000,72.333333
3,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,268,40.0,207.0,747.0,0.333333,49.500000
4,AAA,2013J,38053,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,-110.0,268,70.0,365.0,1271.0,0.500000,74.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23860,GGG,2014J,2620947,F,Scotland,A Level or Equivalent,80-90%,0-35,0,30,Y,Distinction,-23.0,269,14.0,74.0,166.0,0.200000,90.000000
23861,GGG,2014J,2640965,F,Wales,Lower Than A Level,10-20,0-35,0,30,N,Fail,-4.0,269,5.0,19.0,41.0,0.000000,0.000000
23862,GGG,2014J,2645731,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,Distinction,-23.0,269,12.0,83.0,272.0,0.200000,90.000000
23863,GGG,2014J,2648187,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,Pass,-129.0,269,10.0,46.0,99.0,0.300000,66.666667


In [4]:
display(student_assessments)
display(assessments_info)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0
...,...,...,...,...,...
173905,37443,527538,227,0,60.0
173906,37443,534672,229,0,100.0
173907,37443,546286,215,0,80.0
173908,37443,546724,230,0,100.0


,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
...,...,...,...,...,...,...
201,GGG,2014J,37443,CMA,229.0,0.0
202,GGG,2014J,37435,TMA,61.0,0.0
203,GGG,2014J,37436,TMA,124.0,0.0
204,GGG,2014J,37437,TMA,173.0,0.0


In [45]:
assessments = pd.merge(student_assessments, assessments_info, how='left', on='id_assessment')
assessments.dropna(subset = ['score'], inplace=True)
assessments.drop(columns = ['is_banked', 'weight'], inplace=True)
if prediction_window:
    assessments = assessments[assessments['date_submitted'] <= prediction_window]
num_assessments = assessments.groupby(by = index_columns).count().reset_index()
total_assessments = assessments_info.groupby(by=['code_module','code_presentation']).count().reset_index()
total_assessments = total_assessments.drop(columns = ['assessment_type','date','weight'])
total_assessments = total_assessments.rename(columns = {'id_assessment':'total_assessments'})
num_assessments = pd.merge(num_assessments, assess_count, 
                       how = 'left', on = ['code_module','code_presentation'])
num_assessments['id_assessment'] = num_assessments['id_assessment']/num_assessments['total_assessments']
num_assessments.drop(columns = ['total_assessments','date_submitted','score','date','assessment_type'], inplace=True)
num_assessments
avg_score = assessments.groupby(by = index_columns).mean().reset_index()
avg_score.drop(columns = ['date_submitted','id_assessment','date'], inplace=True)

full_registrations = pd.merge(full_registrations, num_assessments, on=index_columns, how='left')
full_registrations = pd.merge(full_registrations, avg_score, on=index_columns, how='left')
full_registrations = full_registrations.fillna(value=0)


,code_module,code_presentation,id_student,id_assessment
0,AAA,2013J,11391,0.833333
1,AAA,2013J,28400,0.833333
2,AAA,2013J,31604,0.833333
3,AAA,2013J,32885,0.833333
4,AAA,2013J,38053,0.833333
...,...,...,...,...
25815,GGG,2014J,2620947,0.900000
25816,GGG,2014J,2645731,0.900000
25817,GGG,2014J,2648187,0.900000
25818,GGG,2014J,2679821,0.200000


In [ ]:
# Assessments
assessments = pd.merge(student_assessments, assessments_info, how='left', on='id_assessment')
assessments.dropna(subset = ['score'], inplace=True)
assessments.drop(columns = ['is_banked', 'weight'], inplace=True)
if prediction_window:
    assessments = assessments[assessments['date_submitted'] <= prediction_window]
num_assessments = assessments.groupby(by = index_columns).count().reset_index()
num_assessments.drop(columns = ['date_submitted','score','date','assessment_type'], inplace=True)

avg_score = assessments.groupby(by = index_columns).mean().reset_index()
avg_score.drop(columns = ['date_submitted','id_assessment','date'], inplace=True)

full_registrations = pd.merge(full_registrations, num_assessments, on=index_columns, how='left')
full_registrations = pd.merge(full_registrations, avg_score, on=index_columns, how='left')
full_registrations = full_registrations.fillna(value=0)